## GAN: MNIST

In [8]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

In [9]:
def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Data

In [10]:
import os
import gzip

def load_mnist_images(data_dir, filename):
    data_path = os.path.join(data_dir, filename)
    with gzip.open(data_path, 'rb') as f:
        data = np.frombuffer(f.read(), np.uint8, offset=16)
    return data.reshape(-1, 28, 28)

def load_mnist_labels(data_dir, filename):
    data_path = os.path.join(data_dir, filename)
    with gzip.open(data_path, 'rb') as f:
        data = np.frombuffer(f.read(), np.uint8, offset=8)
    return data

# data_dir = r"D:\datasets\mnist_11M"   ## windows
data_dir = "/mnt/d/datasets/mnist_11M"  ## wsl

x_train = load_mnist_images(data_dir, "train-images-idx3-ubyte.gz")
y_train = load_mnist_labels(data_dir, "train-labels-idx1-ubyte.gz")
x_test = load_mnist_images(data_dir, "t10k-images-idx3-ubyte.gz")
y_test = load_mnist_labels(data_dir, "t10k-labels-idx1-ubyte.gz")

print(f">> Train images: {x_train.shape}, {x_train.dtype}")
print(f">> Train labels: {y_train.shape}, {y_train.dtype}")
print(f">> Test images:  {x_test.shape}, {x_test.dtype}")
print(f">> Test labels:  {y_test.shape}, {y_test.dtype}")

>> Train images: (60000, 28, 28), uint8
>> Train labels: (60000,), uint8
>> Test images:  (10000, 28, 28), uint8
>> Test labels:  (10000,), uint8


In [11]:
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

class Dataset(torch.utils.data.Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images.reshape(-1, 28, 28, 1)
        self.labels = labels
        self.transform = transform
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)

        label = torch.tensor(label).long()
        return image, label

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5]),
])

train_loader = DataLoader(Dataset(x_train, y_train, transform=transform), 
                          batch_size=64, shuffle=True)
test_loader = DataLoader(Dataset(x_test, y_test, transform=transform), 
                         batch_size=64, shuffle=True)

x, y = next(iter(train_loader))
print(f">> x: {x.shape}, {x.dtype}, min={x.min()}, max={x.max()}")
print(f">> y: {y.shape}, {y.dtype}, min={y.min()}, max={y.max()}")

>> x: torch.Size([64, 1, 28, 28]), torch.float32, min=-1.0, max=1.0
>> y: torch.Size([64]), torch.int64, min=0, max=9


### Modeling

In [12]:
class Generator(nn.Module):
    def __init__(self, input_size, output_size, bias=True):
        super().__init__()
        self.generator = nn.Sequential(
            nn.Linear(input_size, 256, bias=bias),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 256, bias=bias),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 256, bias=bias),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Linear(256, output_size, bias=bias),)

    def forward(self, x):
        x = self.generator(x)
        x = x.view(-1, 1, 28, 28)
        return torch.tanh(x)

class Discriminator(nn.Module):
    def __init__(self, input_size, output_size, bias=True):
        super().__init__()
        self.discriminator = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_size, 256, bias=bias),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 256, bias=bias),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 256, bias=bias),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, output_size, bias=bias),)

    def forward(self, x):
        x = self.discriminator(x)
        return torch.sigmoid(x)

### Training

In [13]:
import sys
from torchvision.utils import save_image
from tqdm import tqdm

## Hyperparameters
set_seed(42)
n_epochs = 50
learning_rate = 2e-4
step_size = 5
noise_size = 64

bias = True
n_outputs = 100
output_name = "gan_mlp_4layer"

## Modeling
modelG = Generator(input_size=noise_size, output_size=28*28, bias=bias).to(device)
modelD = Discriminator(input_size=28*28, output_size=1, bias=bias).to(device)

loss_fn = nn.BCELoss()
optimizerD = optim.Adam(modelD.parameters(), lr=learning_rate, betas=(0.5, 0.999))
optimizerG = optim.Adam(modelG.parameters(), lr=learning_rate, betas=(0.5, 0.999))

## Training
fixed_noise = torch.randn(n_outputs, noise_size).to(device)
output_dir = '/mnt/d/github/lectures-1/test/mnist/output_gan'
output_images = modelG(fixed_noise)
save_image(output_images, os.path.join(output_dir, f"{output_name}_0.png"), nrow=10)

In [14]:
for epoch in range(1, n_epochs + 1):
    with tqdm(train_loader, leave=False, file=sys.stdout, dynamic_ncols=True, ascii=True) as pbar:
        train_loss_r, train_loss_f, train_loss_g = 0, 0, 0
        for i, (real_images, _) in enumerate(pbar):
            batch_size = len(real_images)
            real_labels = torch.ones((batch_size, 1)).to(device)
            fake_labels = torch.zeros((batch_size, 1)).to(device)
            noise = torch.randn(batch_size, noise_size).to(device)
            real_images = real_images.to(device)
            fake_images = modelG(noise)

            ## Training Discriminator
            pred_r = modelD(real_images)
            loss_r = loss_fn(pred_r, real_labels)
            loss_r.backward()

            pred_f = modelD(fake_images.detach())
            loss_f = loss_fn(pred_f, fake_labels)
            loss_f.backward()

            optimizerD.step()
            optimizerD.zero_grad()

            # Training Generator
            pred_g = modelD(fake_images)
            loss_g = loss_fn(pred_g, real_labels)
            loss_g.backward()

            optimizerG.step()
            optimizerG.zero_grad()
            
            train_loss_r += loss_r.item()
            train_loss_f += loss_f.item()
            train_loss_g += loss_g.item()

            desc = f"[{epoch:3d}/{n_epochs}] loss_r: {train_loss_r/(i + 1):.2e} " \
                   f"loss_f: {train_loss_f/(i + 1):.2e} loss_g: {train_loss_g/(i + 1):.2e}"

            if i % 10 == 0:
                pbar.set_description(desc)

        if epoch % step_size == 0:
            print(desc)
            output_images = modelG(fixed_noise)
            save_image(output_images, os.path.join(output_dir, f"{output_name}_{epoch}.png"), nrow=10)

[  5/50] loss_r: 3.02e-01 loss_f: 1.04e+00 loss_g: 4.89e-01                                                    
[ 10/50] loss_r: 3.27e-01 loss_f: 1.06e+00 loss_g: 4.70e-01                                                    
[ 15/50] loss_r: 3.40e-01 loss_f: 1.07e+00 loss_g: 4.60e-01                                                    
[ 20/50] loss_r: 3.51e-01 loss_f: 1.07e+00 loss_g: 4.52e-01                                                     
[ 25/50] loss_r: 3.57e-01 loss_f: 1.08e+00 loss_g: 4.46e-01                                                    
[ 30/50] loss_r: 3.60e-01 loss_f: 1.08e+00 loss_g: 4.44e-01                                                    
[ 35/50] loss_r: 3.62e-01 loss_f: 1.08e+00 loss_g: 4.42e-01                                                    
[ 40/50] loss_r: 3.63e-01 loss_f: 1.08e+00 loss_g: 4.42e-01                                                    
[ 45/50] loss_r: 3.62e-01 loss_f: 1.08e+00 loss_g: 4.42e-01                                            